# 🦌 ELK App 🚀

This is a high-level widget for interacting with ELK Transformers and ELK Diagrams

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import importnb
import ipywidgets
import ipywidgets as W

import ipyelk
import ipyelk.tools
from ipyelk.tools import Selection, ToggleCollapsedTool

In [ ]:
with importnb.Notebook():
    from __02_Transformer import a_hierarchical_elk_example

In [ ]:
def an_elk_app_example(elk=None):
    elk = elk or a_hierarchical_elk_example()
    collapser = elk.get_tool(ipyelk.tools.ToggleCollapsedTool)
    toggle = ipywidgets.Button(description="Toggle")
    toggle.on_click(collapser.handler)

    box = ipywidgets.VBox(
        [
            ipywidgets.HBox([
                ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"),
                toggle,
            ]),
            elk,
        ],
        layout=dict(height="100%"),
    )
    return box, elk


def setup_control_overlay(elk):
    select_tool = elk.get_tool(Selection)
    toggle_tool = elk.get_tool(ToggleCollapsedTool)
    empty = W.VBox()

    def check_icon(*args):
        el = list(select_tool.elements())[0]
        children = getattr(el, "children", [])
        test = any(c.properties.hidden for c in el.children)
        if test:
            icon = "plus"
        else:
            icon = "minus"
        toggle_tool.ui.icon = icon
        toggle_tool.ui.description = ""
        toggle_tool.ui.layout.width = "35px"
        return icon

    def on_select(change):
        for el in select_tool.elements():
            children = getattr(el, "children", [])
            if children:
                elk.view.control_overlay.children = [toggle_tool.ui]
                check_icon()
            else:
                elk.view.control_overlay.children = [empty]

    select_tool.observe(on_select, "ids")
    toggle_tool.on_run(check_icon)

In [ ]:
if __name__ == "__main__":
    box, elk = an_elk_app_example()
    setup_control_overlay(elk)
    display(box)

#### Command Buttons can be customized to tailor the look and feel

In [ ]:
def an_elk_app_with_a_custom_tool_button():
    box, elk = an_elk_app_example()
    fit = elk.get_tool(ipyelk.tools.FitTool).ui
    fit.icon = "arrows-alt"
    fit.tooltip = "Fit Diagram"
    fit.layout.width = "5em"
    fit.description = ""
    return box, elk

In [ ]:
if __name__ == "__main__":
    display(an_elk_app_with_a_custom_tool_button()[1])

#### Demonstrating adding close button

Adding a callable to an `Elk` Application's Toolbar `on_close` will result in a close
button appearing in the toolbar which will fire that function when pressed.

In [ ]:
def an_elk_app_with_custom_close():
    box, elk = an_elk_app_example()

    out = ipywidgets.Output()
    with out:
        print("  ┌ Press the close button!\n  │\n  │\n  │\n◀─╯")

    def simulating_close():
        with out:
            print("─▸Toolbar close button pressed!")

    elk.toolbar.on_close = simulating_close
    return ipywidgets.HBox([box, out]), elk

In [ ]:
if __name__ == "__main__":
    display(an_elk_app_with_custom_close()[1])

## Custom Styling For Collaping Hierarchical Example

The style dictionary takes the first key and adds a parent selector based on the current
`StyleWidget`'s id. This helps to namespace the css selectors but also means that the
leading space in the style dictionary keys is intentional and is a descendant selector
from the root.

The `.slack-port` and `.slack-edge` are custom css classes applied by the XELK
transformer when collapsing a node causes either a source or destination of a tunneling
edges to disapear requiring the introduction of a new edge and port at a higher level.

In [ ]:
def a_styled_elk_app_example():
    box, elk = an_elk_app_example()
    elk.style = {
        " rect": {"opacity": ".75", "transition": "all 0.2s"},
        " .slack-port>rect": {
            "fill": "red",
            "opacity": ".25",
            "transition": "all 0.2s",
        },
        " .slack-edge>path": {
            "stroke": "red",
            "opacity": ".25",
            "stroke-dasharray": "4",
            "transition": "all 0.2s",
            "stroke-width": "2px",
        },
        " .slack-edge>path.edge.arrow": {"fill": "red", "opacity": ".25"},
    }
    return box, elk

In [ ]:
if __name__ == "__main__":
    box, elk = a_styled_elk_app_example()
    display(elk)

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).